## Azure Machine Learning的实验

### 支持Azure Machine Learning

#### Library
在conda yml文件中，一定要包含Azure Machine Learning的Python SDK `azureml-sdk`。

#### 数据集(以自变量提交)
学习数据在azure machine learning dataset (or datastore) 中被注册。在`ScriptRunConfig`中输入` Dataset .as_named_input('input').as_mount()`中，关于Dataset的信息、名称、以下的任一提供方法的信息。
- `as_mount()` : 安装文件
- `as_download()` : 下载文件
    
#### 量度记录
在模型训练过程中，可以记录超参数和模型精度等量度。也可以将各实验的记录横向串起来进行可视化比较。
- `run = Run.get_context()` : run对象的生成
- `run.log("xxx", xxx)`  : 作为实验的量度记录

In [ ]:
pip install azureml


In [ ]:
pip install azureml.core

In [ ]:
pip show azureml-sdk

In [ ]:
pip install 'azureml-sdk[notebooks]'

In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
import os
print(os.getcwd())

### Database的获取

In [ ]:
dataset = Dataset.get_by_name(ws, name='cifar10')

### 设定实验名

In [ ]:
experiment = Experiment(workspace=ws, name='3-aml-run')

### 学习脚本的执行设定

In [ ]:

config = ScriptRunConfig(source_directory='./code/pytorch-cloud',
                         script='train.py',
                         compute_target='gpucluster',  # Compute Cluster的名称
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_download(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])

### 对Environment的调用和执行设定的输入

In [ ]:
env = Environment.get(ws, "pytorch-env")

In [ ]:
config.run_config.environment = env

### 执行和结果确认

In [ ]:
run = experiment.submit(config)

In [ ]:
# Jupyter Widgets
RunDetails(run).show()

In [ ]:
# 文本输出
run.wait_for_completion(show_output=True)

In [ ]:
from azureml.core import Model
model = run.register_model(model_name='cifar10_net_4',
                           model_path='outputs/cifar10_net.pt',
                           model_framework="PyTorch",
                           model_framework_version="1.8.1",
                           tags={'area': 'cifar10'},
                           )
print(model.name, model.id, model.version, sep='\t')

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice


# 在推理设置中，设置评分脚本和环境
inference_config = InferenceConfig(entry_script="pytorch_score.py", environment = env)

# 设置部署配置信息
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

In [ ]:
# 设置模型、推理和部署配置，以及web服务名称和部署位置
cifar_service = Model.deploy(
    workspace = ws,
    name = "cifar-service",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config)

In [ ]:
cifar_service.wait_for_deployment(show_output = True)

**参考信息**
- [配置和提交训练运行](https://docs.microsoft.com/zh-cn/azure/machine-learning/how-to-set-up-training-targets)